## IMPORTAR LIBRERIAS

In [20]:
import pandas as pd
import numpy as np
import os
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')

## Cargar archivos

In [ ]:
### Características Equipos

equipos = pd.read_csv('../data/Caracteristicas_Equipos.csv')
equipos_df = pd.DataFrame(equipos)

In [21]:
### Historicos Ordenes

ordenes = pd.read_csv('../data/Historicos_Ordenes.csv')
ordenes_df = pd.DataFrame(ordenes)

In [ ]:
### Registros Condiciones

condiciones = pd.read_csv('../data/Registros_Condiciones.csv')
condiciones_df = pd.DataFrame(condiciones)

### LIMPIEZA Y PERFILADO DE CARACTERISTICAS EQUIPOS

In [ ]:
# Mostrar las primeras filas de cada dataset
equipos_df.head()

In [ ]:
# Revisar info del df
equipos_df.info()

In [ ]:
# Datos vacios
print('Datos vacios en Caracteristicas_Equipos.csv\n',equipos_df.isna().sum())

In [ ]:
# Extremos
z_scores = (equipos_df-equipos_df.mean(numeric_only=True)) / \
    equipos_df.std(numeric_only=True)
z_scores_abs = z_scores.apply(np.abs)
print(tabulate(z_scores_abs, headers='keys'))

In [ ]:
umbral = 3

out_mask = ~z_scores[z_scores_abs > umbral].isna()
print('\nOutliers per column:\n')
print(out_mask.sum())

In [ ]:
# Mostrar todas las filas duplicadas
print('\nSumatorio duplicados en sales', equipos_df.duplicated().sum())
equipos_df[equipos_df.duplicated(keep=False)]

In [ ]:
# Elimino filas duplicadas

equipos_df_clean = equipos_df.drop_duplicates(keep='first')
print('\nSumatorio duplicados en equipos_df:', equipos_df_clean.duplicated().sum())


In [ ]:
equipos_df_clean

In [ ]:
equipos_df_clean.info()

In [ ]:
# Cardinalidad

# Calcular el sumatorio de valores para cada columna
sum_values_sales = equipos_df_clean.count()

# Calcular el sumatorio de valores únicos para cada columna
unique_sum_sales = equipos_df_clean.nunique()

# Crear un nuevo DataFrame para mostrar ambos sumatorios
result_df_equipos = pd.DataFrame({
    'Sumatorio de valores': sum_values_sales,
    'Sumatorio de valores únicos': unique_sum_sales
})

print(result_df_equipos)

In [ ]:
# Columnas a 'category'

equipos_df_clean[['Tipo_Equipo', 'Fabricante', 'Modelo']] = equipos_df_clean[[
    'Tipo_Equipo', 'Fabricante', 'Modelo']].astype('category')

equipos_df_clean.info()

In [ ]:
equipos_df_clean.describe()

In [ ]:
equipos_df_clean.describe(include='category')

In [ ]:
# Existen equipos con Potencia menor de 0

print("\nEquipos con potencia <0 kW:", len(equipos_df_clean[equipos_df_clean['Potencia_kW'] < 0]))
equipos_df_clean[equipos_df_clean['Potencia_kW'] < 0]

In [ ]:
# Value_counts te muestra cuantos valores hay de cada valor único en la columna selecionada

print(equipos_df_clean[['Tipo_Equipo', 'Fabricante', 'Modelo']].value_counts())

In [ ]:
# Frecuencias en equipos

for col in equipos_df_clean.columns:
    print('\n- Frecuencias para "{0}"'.format(col), '\n')
    print(equipos_df_clean[col].value_counts())

In [ ]:
# Correlación

corr_equipos = equipos_df_clean.corr('pearson', numeric_only=True)
corr_equipos

In [ ]:
# Correlación

corr_equipos[(np.abs(corr_equipos) >= 0.7) & (np.abs(corr_equipos) >= 0.7)]

In [ ]:
# Sesgo

skw_equipos = equipos_df_clean.skew(numeric_only=True)
skw_equipos

In [ ]:
# Sesgo sales

skw_equipos[np.abs(skw_equipos) > 2]

In [ ]:
# Kurtosis

kurt_equipos = equipos_df_clean.kurt(numeric_only=True)
kurt_equipos

In [ ]:
# Kurtosis

kurt_equipos[np.abs(kurt_equipos) > 3]

In [ ]:
# Evaluar la unicidad de posibles claves primarias

# Crear una lista para almacenar los resultados
unike_keys_equipos = []

# Iterar sobre cada columna del DataFrame
for column in equipos_df_clean.columns:
    # Verificar si la columna tiene valores únicos
    if equipos_df_clean[column].is_unique:
        unike_keys_equipos.append(column)

# Mostrar las posibles claves primarias
print("Posibles claves primarias en equipos_df_clean:")
print(unike_keys_equipos)

### LIMPIEZA Y PERFILADO DE HISTORICOS ORDENES

In [22]:
# Mostrar las primeras filas de cada dataset
ordenes_df.head()

,ID_Orden,ID_Equipo,Fecha,Tipo_Mantenimiento,Costo_Mantenimiento,Duracion_Horas,Ubicacion
0,1,465,2020-01-01 00:00:00,Correctivo,1405.20,47,Planta Norte
1,2,418,2020-01-01 01:00:00,Correctivo,5922.92,38,Planta Norte
2,3,228,2020-01-01 02:00:00,Correctivo,7119.05,34,Planta Oeste
3,4,300,2020-01-01 03:00:00,Correctivo,3488.33,38,Planta Oeste
4,5,325,2020-01-01 04:00:00,Correctivo,5611.37,15,Planta Este


In [23]:
# Revisar info del df
ordenes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10020 entries, 0 to 10019
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID_Orden             10020 non-null  int64  
 1   ID_Equipo            10020 non-null  int64  
 2   Fecha                10020 non-null  object 
 3   Tipo_Mantenimiento   10020 non-null  object 
 4   Costo_Mantenimiento  9970 non-null   float64
 5   Duracion_Horas       10020 non-null  int64  
 6   Ubicacion            10020 non-null  object 
dtypes: float64(1), int64(3), object(3)
memory usage: 548.1+ KB


In [24]:
# Datos vacios
print('Datos vacios en Historicos_Ordenes.csv\n',ordenes_df.isna().sum())

Datos vacios en Historicos_Ordenes.csv
 ID_Orden                0
ID_Equipo               0
Fecha                   0
Tipo_Mantenimiento      0
Costo_Mantenimiento    50
Duracion_Horas          0
Ubicacion               0
dtype: int64


In [36]:
# Extremos
z_scores = (ordenes_df-ordenes_df.mean(numeric_only=True)) / \
    ordenes_df.std(numeric_only=True)
z_scores_abs = z_scores.apply(np.abs)
print(tabulate(z_scores_abs, headers='keys'))

         Costo_Mantenimiento    Duracion_Horas    Fecha    ID_Equipo     ID_Orden    Tipo_Mantenimiento    Ubicacion
-----  ---------------------  ----------------  -------  -----------  -----------  --------------------  -----------
    0            1.26138             1.69728        nan   1.48696     1.72493                       nan          nan
    1            0.333379            1.03132        nan   1.16104     1.72459                       nan          nan
    2            0.755613            0.735331       nan   0.156532    1.72424                       nan          nan
    3            0.526032            1.03132        nan   0.342758    1.7239                        nan          nan
    4            0.223402            0.670596       nan   0.516122    1.72355                       nan          nan
    5            1.06534             0.587339       nan   1.37601     1.7232                        nan          nan
    6            1.29543             1.41056        nan   0.1149

In [37]:
umbral = 3

out_mask = ~z_scores[z_scores_abs > umbral].isna()
print('\nOutliers per column:\n')
print(out_mask.sum())


Outliers per column:

Costo_Mantenimiento    0
Duracion_Horas         0
Fecha                  0
ID_Equipo              0
ID_Orden               0
Tipo_Mantenimiento     0
Ubicacion              0
dtype: int64


In [25]:
# Busco filas que tengan el mismo ID_Equipo + Duracion_Horas que la fila con Costo_Mantenimiento vacío y además muestro al final su sumatorio

# Filtrar filas con Costo_Mantenimiento vacío
filas_vacias = ordenes_df[ordenes_df['Costo_Mantenimiento'].isna()]

# Inicializar contador de coincidencias
nan_coincidencias = 0

# Buscar coincidencias en todo el DataFrame
for index, row in filas_vacias.iterrows():
    coincidencias1 = ordenes_df[(ordenes_df['ID_Equipo'] == row['ID_Equipo']) & 
                               (ordenes_df['Duracion_Horas'] == row['Duracion_Horas'])]
    if len(coincidencias1) > 1:
        print(tabulate(coincidencias1, headers='keys'))
        nan_coincidencias += 1

# Mostrar el sumatorio de las filas NaN que han coincidido
print(f'\nSumatorio de filas NaN que han coincidido: {nan_coincidencias}')

        ID_Orden    ID_Equipo  Fecha                Tipo_Mantenimiento      Costo_Mantenimiento    Duracion_Horas  Ubicacion
----  ----------  -----------  -------------------  --------------------  ---------------------  ----------------  -----------
1228        1229          348  2020-02-21 04:00:00  Preventivo                           nan                   15  Planta Este
4424        4425          348  2020-07-03 08:00:00  Correctivo                          9642.19                15  Planta Este
        ID_Orden    ID_Equipo  Fecha                Tipo_Mantenimiento      Costo_Mantenimiento    Duracion_Horas  Ubicacion
----  ----------  -----------  -------------------  --------------------  ---------------------  ----------------  ------------
1437        1438          128  2020-02-29 21:00:00  Preventivo                          9674.38                 1  Planta Oeste
1468        1469          128  2020-03-02 04:00:00  Preventivo                           nan                    1

In [26]:
# Primero sustituyo el valor de Costo_Mantenimiento vacío por el valor de la línea que coincida con ID_Equipo + Duración_Horas + Ubicación

ordenes_df_clean = pd.DataFrame(ordenes_df)

# Filtrar filas con Costo_Mantenimiento vacío
filas_vacias = ordenes_df_clean[ordenes_df_clean['Costo_Mantenimiento'].isna()]

# Buscar coincidencias en todo el DataFrame y reemplazar valores vacíos
for index, row in filas_vacias.iterrows():
    coincidencias2 = ordenes_df_clean[(ordenes_df_clean['ID_Equipo'] == row['ID_Equipo']) & 
                                (ordenes_df_clean['Duracion_Horas'] == row['Duracion_Horas']) &
                                (ordenes_df_clean['Ubicacion'] == row['Ubicacion']) & 
                                (ordenes_df_clean['Costo_Mantenimiento'].notna())]
    if not coincidencias2.empty:
        # Reemplazar el valor vacío en ordenes_df_clean con el valor de la primera coincidencia encontrada
        ordenes_df_clean.at[index, 'Costo_Mantenimiento'] = coincidencias2.iloc[0]['Costo_Mantenimiento']


In [27]:
# Segundo sustituyo el valor de Costo_Mantenimiento vacío por el valor de la línea que coincida ID_Equipo + Duración_Horas

# Filtrar filas con Costo_Mantenimiento vacío
filas_vacias = ordenes_df_clean[ordenes_df_clean['Costo_Mantenimiento'].isna()]

# Buscar coincidencias en todo el DataFrame y reemplazar valores vacíos
for index, row in filas_vacias.iterrows():
    coincidencias3 = ordenes_df_clean[(ordenes_df_clean['ID_Equipo'] == row['ID_Equipo']) & 
                               (ordenes_df_clean['Duracion_Horas'] == row['Duracion_Horas']) &
                               (ordenes_df_clean['Costo_Mantenimiento'].notna())]
    if not coincidencias3.empty:
        # Reemplazar el valor vacío con el valor de la primera coincidencia encontrada
        ordenes_df_clean.at[index, 'Costo_Mantenimiento'] = coincidencias3.iloc[0]['Costo_Mantenimiento']



In [28]:
# Vuelvo a revisar los datos vacíos donde ya he sustituidos los anteriores reduciendo en 17 los valores vacíos
print('Datos vacios en Historicos_Ordenes.csv\n',ordenes_df_clean.isna().sum())

Datos vacios en Historicos_Ordenes.csv
 ID_Orden                0
ID_Equipo               0
Fecha                   0
Tipo_Mantenimiento      0
Costo_Mantenimiento    33
Duracion_Horas          0
Ubicacion               0
dtype: int64


In [ ]:
# Sustituyo valores vacíos por su media

# Calcular la media de la columna Costo_Mantenimiento
mean_costo_mantenimiento = ordenes_df_clean['Costo_Mantenimiento'].mean()

# Sustituir los valores vacíos por la media
ordenes_df_clean['Costo_Mantenimiento'].fillna(mean_costo_mantenimiento, inplace=True)

# Vuelvo a revisar los datos vacíos donde ya he sustituidos los anteriores reduciendo en 17 los valores vacíos
print('Datos vacios en Historicos_Ordenes.csv\n',ordenes_df_clean.isna().sum())

Datos vacios en Historicos_Ordenes.csv
 ID_Orden               0
ID_Equipo              0
Fecha                  0
Tipo_Mantenimiento     0
Costo_Mantenimiento    0
Duracion_Horas         0
Ubicacion              0
dtype: int64


In [ ]:
# Vuelvo a revisar info del df
ordenes_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10020 entries, 0 to 10019
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID_Orden             10020 non-null  int64  
 1   ID_Equipo            10020 non-null  int64  
 2   Fecha                10020 non-null  object 
 3   Tipo_Mantenimiento   10020 non-null  object 
 4   Costo_Mantenimiento  10020 non-null  float64
 5   Duracion_Horas       10020 non-null  int64  
 6   Ubicacion            10020 non-null  object 
dtypes: float64(1), int64(3), object(3)
memory usage: 548.1+ KB


In [33]:
# Mostrar todas las filas duplicadas
print('\nSumatorio duplicados en ordenes', ordenes_df_clean.duplicated().sum())
ordenes_df_clean[ordenes_df_clean.duplicated(keep=False)]


Sumatorio duplicados en ordenes 20


,ID_Orden,ID_Equipo,Fecha,Tipo_Mantenimiento,Costo_Mantenimiento,Duracion_Horas,Ubicacion
0,1,465,2020-01-01 00:00:00,Correctivo,1405.20,47,Planta Norte
1,2,418,2020-01-01 01:00:00,Correctivo,5922.92,38,Planta Norte
2,3,228,2020-01-01 02:00:00,Correctivo,7119.05,34,Planta Oeste
3,4,300,2020-01-01 03:00:00,Correctivo,3488.33,38,Planta Oeste
4,5,325,2020-01-01 04:00:00,Correctivo,5611.37,15,Planta Este
5,6,449,2020-01-01 05:00:00,Preventivo,7996.45,32,Planta Norte
6,7,234,2020-01-01 06:00:00,Preventivo,8648.28,5,Planta Este
7,8,487,2020-01-01 07:00:00,Correctivo,2329.24,6,Planta Sur
8,9,191,2020-01-01 08:00:00,Correctivo,1743.03,33,Planta Este
9,10,444,2020-01-01 09:00:00,Preventivo,8899.57,30,Planta Norte


In [35]:
# Elimino filas duplicadas

ordenes_df_cleaned = ordenes_df_clean.drop_duplicates(keep='first')
print('\nSumatorio duplicados en ordenes_df_clean:', ordenes_df_cleaned.duplicated().sum())


Sumatorio duplicados en ordenes_df_clean: 0


In [39]:
# Vuelvo a revisar info del df
ordenes_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID_Orden             10000 non-null  int64  
 1   ID_Equipo            10000 non-null  int64  
 2   Fecha                10000 non-null  object 
 3   Tipo_Mantenimiento   10000 non-null  object 
 4   Costo_Mantenimiento  10000 non-null  float64
 5   Duracion_Horas       10000 non-null  int64  
 6   Ubicacion            10000 non-null  object 
dtypes: float64(1), int64(3), object(3)
memory usage: 625.0+ KB
